In [1]:
import tweepy
import json
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# Authorization Codes
consumer_key = "ZCVjbgfaSGlrJVh7T2cqIFfMR"
consumer_secret = "yah2rFbZE1qX9n1MDaqObT1PfCOSYlvxIyXYuCO4ZrSUfSvjEC"
access_key = "781555439183859712-tS4fKwo03ujDtNL0UibVIXauXRFdFvs"
access_secret = "3YrATCHVvP2atvFs4M6VJ0dFvsdE3BBKoZ2OcvKdqPzJp"

# OAuth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# API Call
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser())

In [2]:
# Test API by pulling data from Donald Trumps twitter
data = api.user_timeline(id='realDonaldTrump', count = 200)  #Example:@realDonaldTrump
for i in range(10):
    print([i],data[i]['text'])

[0] I will be announcing my decision on the Paris Accord over the next few days. MAKE AMERICA GREAT AGAIN!
[1] Hopefully Republican Senators, good people all, can quickly get together and pass a new (repeal &amp; replace) HEALTHCARE bill. Add saved $'s.
[2] Kathy Griffin should be ashamed of herself. My children, especially my 11 year old son, Barron, are having a hard time with this. Sick!
[3] ...case against him &amp; now wants to clear his name by showing "the false or misleading testimony by James Comey, John Brennan..." Witch Hunt!
[4] So now it is reported that the Democrats, who have excoriated Carter Page about Russia, don't want him to testify. He blows away their....
[5] Who can figure out the true meaning of "covfefe" ???  Enjoy!
[6] RT @foxandfriends: Jared Kushner didn't suggest Russian communications channel in meeting, source says https://t.co/nF6bM1FEt1
[7] RT @Scavino45: President Trump pays respects and  delivers #MemorialDay remarks at Arlington National Cemetery. ht

In [8]:
search_query =  '"Ethereum" -filter:retweets'
data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed')
all_data = list(data.values())[1]

all_data[1]['text']

'We will begin offering Ethereum trading on https://t.co/1RcNoD3vzA and OKCoin.cn Full details: https://t.co/BOAuH6s09f #eth #etc'

In [10]:
while(len(all_data) <= 1000):
    #time.sleep(5)
    last = all_data[-1]['id']
    data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed', since_id = last)
    all_data += list(data.values())[1][1:]


In [16]:
len(all_data)

1034

In [17]:
tweet = []
number_favourites = []
vs_compound = []
vs_pos = []
vs_neu = []
vs_neg = []

analyzer = SentimentIntensityAnalyzer()